# LL(1) table parsing
Christopher Chen
CBU CSC311 HW 4
Professor Im
2017-10-09


In [59]:
table_string = """	id	*	+	(	)	$
A	CB			CB	
B			+CB		e	e
C	ED			ED	
D		*ED	e		e	e
E	id			(A)
"""
print(table_string)

	id	*	+	(	)	$
A	CB			CB	
B			+CB		e	e
C	ED			ED	
D		*ED	e		e	e
E	id			(A)



In [79]:
class Epsilon():
    """Class to represent Epsilon tokens"""
    def __str__(self):
        return "Epsilon()"
    def __repr__(self):
        return str(self)

# Input sanitization and custom data structure

In [80]:
# string manipulation
lines = [l.strip().split('\t') for l in table_string.strip().split('\n')]
header = lines.pop(0)
symbols = [l.pop(0) for l in lines]

replacement = {
    '':None,
    'e': Epsilon(),
              }
for index, line in enumerate(lines):
    padding = [None]*(len(header)-len(line))
    lines[index] = [replacement.get(i,i) for i in line] + padding

In [81]:
# now we have a 2d array representation of the table
lines

[['CB', None, None, 'CB', None, None],
 [None, None, '+CB', None, Epsilon(), Epsilon()],
 ['ED', None, None, 'ED', None, None],
 [None, '*ED', Epsilon(), None, Epsilon(), Epsilon()],
 ['id', None, None, '(A)', None, None]]

In [84]:
# create a nested mapping representation of table
from pprint import pprint
table = {}
for i in zip(symbols, lines):
    table[i[0]] = dict(zip(header,i[1]))
pprint(table)

{'A': {'$': None, '(': 'CB', ')': None, '*': None, '+': None, 'id': 'CB'},
 'B': {'$': Epsilon(),
       '(': None,
       ')': Epsilon(),
       '*': None,
       '+': '+CB',
       'id': None},
 'C': {'$': None, '(': 'ED', ')': None, '*': None, '+': None, 'id': 'ED'},
 'D': {'$': Epsilon(),
       '(': None,
       ')': Epsilon(),
       '*': '*ED',
       '+': Epsilon(),
       'id': None},
 'E': {'$': None, '(': '(A)', ')': None, '*': None, '+': None, 'id': 'id'}}


# Time to parse

In [83]:
to_parse = "(x + y) + a * b + c$"
input_stack = [i for i in reversed(to_parse) if i != ' ']
parse_stack = ['$',symbols[0]]


def debug():
    # shenanigans for more readable output.
    print(*map(lambda t: ' '.join(reversed(t)),[parse_stack, input_stack]),sep=' | ')
debug()
# impose hard limit to iterations in case of mistakes
for i in range(100):
    ps = parse_stack.pop()
#     check for exit
    if ps == '$':
        print("success")
        break
    else:

        
#     look at next token
        lookahead = input_stack[-1]
        print(ps,lookahead,'=>',action)
#     pop terminals
        if ps==lookahead or (ps=='id' and lookahead.isalpha()):
            input_stack.pop()
#     popping terminals like zits
        elif ps == lookahead:
            input_stack.pop()
#       case of a production rule on the parse stack
        elif ps in symbols:
#       convert non-epsilon literal to generic id
            if (lookahead not in header) and lookahead != 'e':
                lookahead = 'id'
#           get next parse stack item from LL table
            action = table[ps][lookahead]
            
#           detect invalid input by attempts to access invalid parts of table.
            if action is None:
                print("fail")
                break
#           more popping literals
            elif action =='id' and lookahead == 'id':
                input_stack.pop()
            if not isinstance(action,Epsilon):
                if action == 'id':
                    parse_stack.append(action)
                else:
                    parse_stack.extend(reversed(action))
    debug()



A $ | ( x + y ) + a * b + c $
A ( => Eps()
C B $ | ( x + y ) + a * b + c $
C ( => CB
E D B $ | ( x + y ) + a * b + c $
E ( => ED
( A ) D B $ | ( x + y ) + a * b + c $
( ( => (A)
A ) D B $ | x + y ) + a * b + c $
A x => (A)
C B ) D B $ | x + y ) + a * b + c $
C x => CB
E D B ) D B $ | x + y ) + a * b + c $
E x => ED
id D B ) D B $ | + y ) + a * b + c $
id + => id
D B ) D B $ | + y ) + a * b + c $
D + => id
B ) D B $ | + y ) + a * b + c $
B + => Epsilon()
+ C B ) D B $ | + y ) + a * b + c $
+ + => +CB
C B ) D B $ | y ) + a * b + c $
C y => +CB
E D B ) D B $ | y ) + a * b + c $
E y => ED
id D B ) D B $ | ) + a * b + c $
id ) => id
D B ) D B $ | ) + a * b + c $
D ) => id
B ) D B $ | ) + a * b + c $
B ) => Epsilon()
) D B $ | ) + a * b + c $
) ) => Epsilon()
D B $ | + a * b + c $
D + => Epsilon()
B $ | + a * b + c $
B + => Epsilon()
+ C B $ | + a * b + c $
+ + => +CB
C B $ | a * b + c $
C a => +CB
E D B $ | a * b + c $
E a => ED
id D B $ | * b + c $
id * => id
D B $ | * b + c $
D * => id
* 